# Tratamento Inicial de Dados

Neste tratamento de dados inicial, apenas é feita a **transformação e normalização de dados do tipo string**.  

- **O que é feito:**  
  - Normalização de colunas categóricas (`strip`, `upper`/`lower`)  
  - Remoção de acentos usando `unidecode`  
  - Conversão de colunas para tipo `category` para economizar memória  

- **O que não é feito:**  
  - Tratamento de valores ausentes (NaNs)  
  - Tratamento de outliers  

> Você pode verificar que o **número de NaNs não se altera** após este processo.

No final, o dataset é salvo no formato **pickle** para uso posterior.

In [1]:
import pandas as pd 
from unidecode import unidecode
import numpy as np
import os

In [2]:
dirpath = os.path.join('..','data','raw')
print(os.listdir(dirpath))
data  = [pd.read_csv(os.path.join(dirpath,file),encoding='latin1',sep=';') for file in os.listdir(dirpath)]
data = pd.concat(data)

['acidentes2017_todas_causas_tipos.zip', 'acidentes2018_todas_causas_tipos.zip', 'acidentes2019_todas_causas_tipos.zip', 'acidentes2020_todas_causas_tipos.zip', 'acidentes2021_todas_causas_tipos.zip', 'acidentes2022_todas_causas_tipos.zip', 'acidentes2023_todas_causas_tipos.zip', 'acidentes2024_todas_causas_tipos.zip']


In [3]:
# Normalizo o nomes das colunas
data.columns = data.columns.str.strip().str.lower()
data.columns = [unidecode(col) for col in data.columns]

In [4]:
def is_mixed(col):
    return len(set(type(x) for x in col if pd.notna(x)))>1
# Converte dados mistos em strings, Deveria também tratar possiveis Nans, mas aqui apenas as colunas longitude e latitude tem valores mistos, 
# e como elas não tem Nans, eu apliquei em toda a coluna sem risco
for c in data.columns:
    if is_mixed(data[c]):
        print(f'A coluna {c} for forçada para string')
        data[c] = data[c].astype(str) 

A coluna latitude for forçada para string
A coluna longitude for forçada para string


In [5]:
# Alteração do nome data_inversa para data, e limpeza do nome das colunas
data.columns = data.columns.str.strip().str.lower()
data.rename(columns={'data_inversa':'data'},inplace=True)
# é possivel obter horario e dia da semana a partir da coluna data, então eliminei colunas horario e dia da semana
data['data'] = pd.to_datetime(data['data']+' '+data['horario'])
data.drop(['horario','dia_semana'],axis=1,inplace=True)

In [6]:
# Testa se todos os valores(excluindo Nans) são inteiros, se sim força conversão para Int64
def is_integer(col):
    notna = col[col.notna()]
    return (notna % 1 == 0).all()
integers = ['id','pesid','idade','ilesos','feridos_leves','feridos_graves','mortos','ano_fabricacao_veiculo','id_veiculo']
for i in integers:
    if is_integer(data[i]):
        print(f'A coluna {i} foi forçada para Int64')
        data[i] = data[i].astype('Int64')

A coluna id foi forçada para Int64
A coluna pesid foi forçada para Int64
A coluna idade foi forçada para Int64
A coluna ilesos foi forçada para Int64
A coluna feridos_leves foi forçada para Int64
A coluna feridos_graves foi forçada para Int64
A coluna mortos foi forçada para Int64
A coluna ano_fabricacao_veiculo foi forçada para Int64
A coluna id_veiculo foi forçada para Int64


In [7]:
# Transformo o id e pesid para inteiros
data['id'] = data['id'].astype('Int64')
data['pesid'] = data['pesid'].astype('Int64')

In [8]:
# Normalização de colunas categoricas que são strings
cat_upper = ['uf',
            'municipio',
            'delegacia',
            'regional',
            'uop'
]
for cat in cat_upper:
    data[cat] = data[cat].str.strip().str.upper()
    data[cat] = data[cat].map(lambda s: unidecode(s) if pd.notna(s) else s)
    data[cat] = data[cat].astype('category')
    print(f'A coluna {cat} foi convertida para category(uppercase)')
cat_lower = ['causa_principal',
            'causa_acidente',
            'tipo_acidente',
            'classificacao_acidente',
            'fase_dia',
            'sentido_via',
            'condicao_metereologica',
            'tipo_pista',
            'tracado_via',
            'uso_solo',
            'sexo',
            'tipo_envolvido',
            'estado_fisico',
            'tipo_veiculo',
            'marca'
]
for cat in cat_lower:
    data[cat] = data[cat].str.strip().str.lower()
    data[cat] = data[cat].map(lambda s: unidecode(s) if pd.notna(s) else s)
    data[cat] = data[cat].astype('category')
    print(f'A coluna {cat} foi convertida para category(lowercase)')


A coluna uf foi convertida para category(uppercase)
A coluna municipio foi convertida para category(uppercase)
A coluna delegacia foi convertida para category(uppercase)
A coluna regional foi convertida para category(uppercase)
A coluna uop foi convertida para category(uppercase)
A coluna causa_principal foi convertida para category(lowercase)
A coluna causa_acidente foi convertida para category(lowercase)
A coluna tipo_acidente foi convertida para category(lowercase)
A coluna classificacao_acidente foi convertida para category(lowercase)
A coluna fase_dia foi convertida para category(lowercase)
A coluna sentido_via foi convertida para category(lowercase)
A coluna condicao_metereologica foi convertida para category(lowercase)
A coluna tipo_pista foi convertida para category(lowercase)
A coluna tracado_via foi convertida para category(lowercase)
A coluna uso_solo foi convertida para category(lowercase)
A coluna sexo foi convertida para category(lowercase)
A coluna tipo_envolvido foi con

In [9]:
data['km'] = data['km'].str.replace(',','.').astype('float')
data['latitude'] = data['latitude'].str.strip().str.replace(',','.').astype(float)
data['longitude'] = data['longitude'].str.strip().str.replace(',','.').astype(float)
data['br'] = data['br'].astype('category')
data['ordem_tipo_acidente'] = data['ordem_tipo_acidente'].astype('category')

In [10]:
data.to_pickle(os.path.join('..','data','processed','datatran.pkl'))